<a href="https://colab.research.google.com/github/srehaag/legal_info_tech_w25/blob/main/Solutions_Module2_assignment_w25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2: Assignment Solutions

There isn't usually just one way to solve a coding problem. This notebook offers some examples of how you might approach the assignment questions -- but these are not the only possible approaches.

Recall that modules are graded as a compoent of particiaption grades on a fail, pass, high pass basis. Students receive a high pass if they make a meaningful effort to complete the module. Pass grades are where students mostly complete the assignment but should have put more effort into the assignment. And fail is where the assignment is not mostly complete and insufficient effort was put into the assignment.

If you would like further feedback on your completed assignment, including invidivualized feedback on your coding, please drop in during the "Study with me" sessions, or reach out by email.


### Beginner Question 1:

Create a text file called hello.txt with the contents:

Hello World!

In [36]:
with open('hello.txt', 'w') as f:
    f.write('Hello World!')

### Beginner Question 2:

Find a course summary (yours or one that you find online) for an Osgoode Hall Law School course in MS Word docx format.

Extract the text of the document into a variable called: course_summary_text

Print the first 5 paragraphs of the course_summary_text variable

In [37]:
!pip install python-docx
from docx import Document

doc = Document("Test Summary.docx")

for para in doc.paragraphs[:5]:
    print(para.text)

Test1
Test2
Test3
Test4
Test5


### Beginner Question 3:

Create a pandas dataframe using data about refugee claims decided in 2019 available at: https://refugeelab.ca/wp-content/uploads/2024/06/2019_RPD_Data.xlsx

Using the dataframe, list the 5 most common countries of persecution in 2019.

In [38]:
import pandas as pd

url = 'https://refugeelab.ca/wp-content/uploads/2024/06/2019_RPD_Data.xlsx'

df = pd.read_excel(url)

df["Country Persecution"].value_counts().head(5)


,count
Country Persecution,
Nigeria,2293
Haiti,1981
Iran,1668
Turkey,1320
Mexico,1309


### Beginner Question 4:

Load the Refugee Law Lab's Social Security Tribunal Bulk Decisions Dataset using [Hugging Face Datasets](https://huggingface.co/datasets/refugee-law-lab/canadian-legal-data). Convert the dataset to a pandas dataframe. Then print the number of SST decisions in this dataset that are available in English (vs French).

You can find additional information about this dataset [here](https://refugeelab.ca/bulk-data).

In [39]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("refugee-law-lab/canadian-legal-data", "SST", split="train")
df = dataset.to_pandas()
print(len(df[df.language == 'en']))

15510


### Intermediate Question 1:

Using the same dataframe about refugee claims in 2019 from Beginner Question 3, filter the dataframe so that it only includes positive and negative decisions (excluding claims that were otherwise resolved).

Positive decisions include the following:


*   Positive
*   Expedited Positive

Negative decisions include the following:

*   Negative
*   Neg. No Cred Basis

Using the filtered dataframe, caculate and print the recognition rate (proportion of positive relative to the sum of positive and negative) for the ten highest volume decision-makers -- listed from highest recognition rate to lowest recognition rate.

In [40]:
import pandas as pd

# Load the data from the Excel file
url = 'https://refugeelab.ca/wp-content/uploads/2024/06/2019_RPD_Data.xlsx'
df = pd.read_excel(url)

# Filter for desired decision explanations
list_of_wanted_results = ['Positive',
                          'Expedited Positive',
                          'Negative',
                          'Neg. No Cred Basis']
df = df[df['Explanation'].isin(list_of_wanted_results)]

# Find the 10 most frequent "Member Fullname" values
top_10_members = df['Member Fullname'].value_counts().head(10).index

# Filter the df to include only rows with those top 10 "Member Fullname" values
df = df[df['Member Fullname'].isin(top_10_members)]

# Calculate the recognition rate for each "Member Fullname" and store in a list
recognition_rates = []
for member in top_10_members:
    member_df = df[df['Member Fullname'] == member]
    positive_count = len(member_df[member_df['Explanation'].isin(['Positive', 'Expedited Positive'])])
    recognition_rate = positive_count / len(member_df)
    recognition_rates.append((member, recognition_rate))

# Sort the list by recognition rate in descending order
recognition_rates.sort(key=lambda item: item[1], reverse=True)

# Print the sorted list
for member, rate in recognition_rates:
    print(f"{member}: {rate}")


RAMSAY, François: 1.0
FIGG, LOIS: 0.9820627802690582
BAFARO, ROBERT: 0.9627118644067797
MORIN, Julie: 0.9548611111111112
PHAM, Hai Yen: 0.9479553903345725
CALISTO AZEVEDO, Mélanie: 0.9377777777777778
RICO, ANA: 0.8864628820960698
CUNDAL, KERRY: 0.8451882845188284
GIBBS, CAROL-ANN: 0.834061135371179
ADHOPIA, PREETI: 0.7853881278538812


In [41]:
# ALTERNATIVE APPROACH:
# Credit to Yuehua Li for this solution

import pandas as pd

# Download the Excel file
url = "https://refugeelab.ca/wp-content/uploads/2024/06/2019_RPD_Data.xlsx"
df = pd.read_excel(url)

# Define lists of positive and negative decision types
positive_decisions = ['Positive', 'Expedited Positive']
negative_decisions = ['Negative', 'Neg. No Cred Basis']

# Step 1: Filter the DataFrame for positive and negative decisions
filtered_df = df[df['Explanation'].isin(positive_decisions + negative_decisions)].copy()

# Step 2: Categorize each decision as 'Positive' or 'Negative'
filtered_df['Decision_Category'] = filtered_df['Explanation'].apply(
    lambda x: 'Positive' if x in positive_decisions else 'Negative'
)

# Step 3: Group by 'Member Fullname' and count decisions
decision_counts = filtered_df.groupby('Member Fullname')['Decision_Category'].value_counts().unstack(fill_value=0)

# Step 4: Calculate recognition rate
decision_counts['Recognition_Rate'] = decision_counts['Positive'] / (decision_counts['Positive'] + decision_counts['Negative'])

# Step 5: Calculate total decisions per decision-maker
decision_counts['Total_Decisions'] = decision_counts['Positive'] + decision_counts['Negative']

# Step 6: Sort by total decisions and select top 10
top_decision_makers = decision_counts.sort_values(by='Total_Decisions', ascending=False).head(10)

# Step 7: Sort the top decision-makers by recognition rate
top_decision_makers_sorted = top_decision_makers.sort_values(by='Recognition_Rate', ascending=False)

# Step 8: Prepare the result for display
result = top_decision_makers_sorted[['Positive', 'Negative', 'Recognition_Rate']].reset_index()
result = result.rename(columns={
    'Member Fullname': 'Decision_Maker',
    'Positive': 'Positive_Decisions',
    'Negative': 'Negative_Decisions',
    'Recognition_Rate': 'Recognition_Rate'
})

# Step 9: Display the results
print("Recognition Rate for Top 10 Decision-Makers:")
result

Recognition Rate for Top 10 Decision-Makers:


Decision_Category,Decision_Maker,Positive_Decisions,Negative_Decisions,Recognition_Rate
0,"RAMSAY, François",238,0,1.000000
1,"FIGG, LOIS",219,4,0.982063
2,"BAFARO, ROBERT",284,11,0.962712
3,"MORIN, Julie",275,13,0.954861
4,"PHAM, Hai Yen",255,14,0.947955
5,"CALISTO AZEVEDO, Mélanie",211,14,0.937778
6,"RICO, ANA",203,26,0.886463
7,"CUNDAL, KERRY",202,37,0.845188
8,"GIBBS, CAROL-ANN",191,38,0.834061
9,"ADHOPIA, PREETI",172,47,0.785388


In [42]:
# ALTERNATIVE APPROACH
# Credit to Aidan Wong

import pandas as pd

url = "https://refugeelab.ca/wp-content/uploads/2024/06/2019_RPD_Data.xlsx"
df = pd.read_excel(url)

positive_decisions = ['Positive', 'Expedited Positive']
negative_decisions = ['Negative', 'Neg. No Cred Basis']

filtered_df = df[df['Explanation'].isin(positive_decisions + negative_decisions)]

decision_maker_counts = filtered_df.groupby('Member Fullname')['Explanation'].value_counts().unstack(fill_value=0)
decision_maker_counts['Total'] = decision_maker_counts.sum(axis=1)
decision_maker_counts['Positive Proportion'] = (
    (decision_maker_counts['Positive'] + decision_maker_counts['Expedited Positive']) /
    decision_maker_counts['Total']
)
top_ten_decision_makers = decision_maker_counts.sort_values(by='Total', ascending=False).head(10)
top_ten_decision_makers = top_ten_decision_makers.sort_values(by='Positive Proportion', ascending=False)

print(top_ten_decision_makers[['Positive Proportion']])

Explanation               Positive Proportion
Member Fullname                              
RAMSAY, François                     1.000000
FIGG, LOIS                           0.982063
BAFARO, ROBERT                       0.962712
MORIN, Julie                         0.954861
PHAM, Hai Yen                        0.947955
CALISTO AZEVEDO, Mélanie             0.937778
RICO, ANA                            0.886463
CUNDAL, KERRY                        0.845188
GIBBS, CAROL-ANN                     0.834061
ADHOPIA, PREETI                      0.785388


### Intermediate Question 2:

Find an online source for the Online News Act, SC 2023, c23.

Programmatically download the text of that Act into a variable called target_statute, and print the first 100 characters that variable.

In [43]:
#!pip install datasets
from datasets import load_dataset

# Load Federal Legislation dataset from RLL
dataset = load_dataset("refugee-law-lab/canadian-legal-data", "LEGISLATION-FED", split="train")
df = dataset.to_pandas()

# get target statute ("Online News Act")
target_statute = df[df['name'] == 'Online News Act'].iloc[0]['unofficial_text']

# print first 100 characters of the target statute
print(target_statute[:100])

# Online News Act

SC 2023, c 23

An Act respecting online communications platforms that make news c


In [44]:
# ALTERNATIVE APPROACH
# Credit to Yuehua Li for this solution

import requests
import xml.etree.ElementTree as ET

url = "https://laws.justice.gc.ca/eng/XML/O-9.3.xml"

try:
    print(f"Fetching XML data from {url}...")
    response = requests.get(url)
    response.raise_for_status()
    print("Successfully fetched the XML data.")

    print("Parsing the XML data...")
    root = ET.fromstring(response.content)
    print("XML parsing completed.")

    target_statute = ""

    for elem in root.iter():
        if elem.text and elem.text.strip():
            target_statute += elem.text.strip() + " "

    if target_statute:
        print("\nFirst 100 characters of the Online News Act:")
        print(target_statute[:100])
    else:
        print("No textual content was extracted from the XML.")

except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching the XML: {e}")
except ET.ParseError as pe:
    print(f"An error occurred while parsing the XML: {pe}")
except Exception as ex:
    print(f"An unexpected error occurred: {ex}")


Fetching XML data from https://laws.justice.gc.ca/eng/XML/O-9.3.xml...
Successfully fetched the XML data.
Parsing the XML data...
XML parsing completed.

First 100 characters of the Online News Act:
C-18 1 44 70-71 Elizabeth II – 1 Charles III 2021-2022-2023 An Act respecting online communications 
